In [9]:
import pickle
import pandas as pd
import numpy as np
from sklearn import metrics

model_cols = ['food', 'shelter', 'water']

In [10]:
test = pd.read_csv("./preprocessed/test.csv")

In [11]:
with open("./pickles/model.pickle", 'rb') as file:
    data = pickle.load(file)

In [12]:
print(data)

{'classifiers': {'request': SVC(C=3.0, kernel='linear'), 'food': SVC(C=3.0, kernel='linear'), 'shelter': SVC(C=3.0, kernel='linear'), 'water': SVC(C=3.0, kernel='linear')}, 'vectorizer': TfidfVectorizer(max_features=100000, ngram_range=(1, 2),
                strip_accents='unicode',
                tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL)>)}


In [13]:
vectorizer = data['vectorizer']
classifiers = data['classifiers']

In [14]:
# Check request accuracy
clf = classifiers['request']

val_x = test.message_stem
val_y = test.request

val_input = vectorizer.transform(val_x)
prediction = clf.predict(val_input)

score = metrics.accuracy_score(prediction, val_y)
report = metrics.classification_report(prediction, val_y)

print(score)
print(report)

0.8984018264840182
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      2222
           1       0.64      0.79      0.70       406

    accuracy                           0.90      2628
   macro avg       0.80      0.85      0.82      2628
weighted avg       0.91      0.90      0.90      2628



In [15]:
test_stripped = test[test.request == 1]

In [16]:
clf = classifiers['food']

val_x = test_stripped.message_stem
val_y = test_stripped.food

val_input = vectorizer.transform(val_x)
prediction = clf.predict(val_input)

score = metrics.accuracy_score(prediction, val_y)
report = metrics.classification_report(prediction, val_y)

print(score)
print(report)

0.9639278557114228
              precision    recall  f1-score   support

           0       1.00      0.95      0.97       344
           1       0.90      0.99      0.94       155

    accuracy                           0.96       499
   macro avg       0.95      0.97      0.96       499
weighted avg       0.97      0.96      0.96       499



In [17]:
clf = classifiers['shelter']

val_x = test_stripped.message_stem
val_y = test_stripped.shelter

val_input = vectorizer.transform(val_x)
prediction = clf.predict(val_input)

score = metrics.accuracy_score(prediction, val_y)
report = metrics.classification_report(prediction, val_y)

print(score)
print(report)

0.9298597194388778
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       376
           1       0.86      0.85      0.86       123

    accuracy                           0.93       499
   macro avg       0.91      0.90      0.91       499
weighted avg       0.93      0.93      0.93       499



In [18]:
clf = classifiers['water']

val_x = test_stripped.message_stem
val_y = test_stripped.water

val_input = vectorizer.transform(val_x)
prediction = clf.predict(val_input)

score = metrics.accuracy_score(prediction, val_y)
report = metrics.classification_report(prediction, val_y)

print(score)
print(report)

0.9779559118236473
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       408
           1       0.92      0.97      0.94        91

    accuracy                           0.98       499
   macro avg       0.95      0.97      0.96       499
weighted avg       0.98      0.98      0.98       499

